# Programación MINLP Ejemplo de Grossmann
### Elaborado por Daniel Ovalle - 201631778
### Asesorado por Jorge Mario Gómez y Camilo Gómez

El objetivo de este notebook es abordar el problema de pooling que expone Grossmann desde la formulación MINLP.

### Sets:
$N=$ Total number of tanks <br>
$A=$ Total number of feasible arcs <br>
$B=$ Blending tanks <br>
$S=$ Supply tanks <br>
$D=$ Demand tanks <br>
$Q=$ Specifications <br>
$T=$ Time periods <br>

### Continuous Variables: 
$F_{nn',t}=$ Flow between tanks $n$ and $n'$ at the end of time $t$<br>
$FD_{d,t}=$ Demand flow from tanks $d$ at time $t$ <br>
$I_{n,t}=$ Inventory in tank $n$ at the end of time $t$ <br>
$C_{q,b,t}=$ Specification $q$ in tank $b$ at the end of time $t$ <br>

### Binary Variables:
$X_{nn',t}=$ Indicates the existence of flow between tanks $n$ and $n'$ at the end of time $t$ <br>

### Parameters:
$I^0_n=$ Initial inventory for tank $n$ <br>
$C^0_{q,b}=$ Initial values for the specifications $q$ in tank $b$ <br>
$F^{IN}_{s,t}=$ Incoming supply flows enter tank $s$ at time $t$ <br>
$C^{IN}_{q,s}=$ Specification $q$ in supply flow to tank $s$ <br>
$\hat{C}^0_{q,r}$ Specification $q$ in source $r$ <br>
$[FD^L_{d,t} , FD^U_{d,t}]=$ Bounds on demand flow from tanks $d$ at time $t$ <br>
$[C^L_{q,d} , C^U_{q,d}]=$ Bounds on specification $q$ in demand tank $d$ <br>
$[I^L_n , I^U_n]=$ Bounds on inventory for tank $n$ <br>
$[F^L_{nn'} , F^U_{nn'}]=$ Bounds on flow between tank $n$ and $n'$ <br>
$\beta^T_s=$ Costs for the supply flow for tank $s$ <br>
$\beta^T_d=$ Prices for demand flow for tank $d$ <br>
$\alpha^N_{nn'}=$ Fixed costs for flow from tank $n$ to tnak $n'$ <br>
$\beta^N_{nn'}=$ Variable costs for flow from tank $n$ to tnak $n'$ <br>

In [1]:
# Import all

import numpy as np
import pandas as pd
import networkx as nx
from gurobipy import *
import matplotlib.pyplot as plt
import time as time
import random as random

In [2]:
# Create Sets
B = [4,5,6,7,8,9,10,11,12,13,14,15]
S = [1,2,3]
D = [16,17,18]
N = S + B + D

B1 = [4,5,6,7,8,9]
B2 = [10,11,12,13,14,15]

Asn = {(s,n):1 for s in S for n in B1}
And = {(n,d):1 for n in B2 for d in D}
Abb = {(b1,b2):1 for b1 in B1 for b2 in B2}
Asd = {(s,d):1 for s in S for d in D}
A = {}
for i in (Asn, Abb, And, Asd): 
    A.update(i)

Q = ['A']
T = list(range(1,7))

# Create Parameters

# Initial conditions
I0_b = {b:0 for b in B}
I0_s = {s:0 for s in S}
I0_d = {d:0 for d in D}
I0_n = {}
for i in (I0_s, I0_b, I0_d): 
    I0_n.update(i)
C0_qb = {(q,b):0 for q in Q for b in B}

# Bounds
IL_n = {n:0 for n in N}
IU_s = {s:0 for s in S}
IU_d = {d:0 for d in D}
IU_b1 = {b1:40 for b1 in B1}
IU_b2 = {b2:30 for b2 in B2}
IU_b = {}
for i in (IU_b1, IU_b2): 
    IU_b.update(i)
IU_n = {}
for i in (IU_s, IU_b, IU_d): 
    IU_n.update(i)

Fmax = 40    
    
FL_nn = {nn:10 for nn in A}
FU_nn = {nn:Fmax for nn in A}

FDL_dt = {(16,1):0, (16,2):0, (16,3):10, (16,4):10, (16,5):10, (16,6):10, (17,1):0, (17,2):0, (17,3):10, (17,4):10, (17,5):10, (17,6):10, (18,1):0, (18,2):0, (18,3):10, (18,4):10, (18,5):10, (18,6):10}
FDU_dt = {(d,t):Fmax for d in D for t in T}

CL_qd = {(q,d):0 for q in Q for d in D}
CU_qd = {('A',16):0.15, ('A',17):0.35, ('A',18):1}

CL_q = {'A':0.05}
CU_q = {'A':0.45}

# Supply Conditions
CIN_qs = {('A',1):0.05, ('A',2):0.30, ('A',3):0.45}
FIN_st = {(1,1):30, (1,2):30, (1,3):30, (1,4):0, (1,5):0, (1,6):0, (2,1):20, (2,2):20, (2,3):20, (2,4):0, (2,5):0, (2,6):0, (3,1):10, (3,2):10, (3,3):10, (3,4):0, (3,5):0, (3,6):0}

# Economic parameters
betaT_s = {s:0 for s in S}
betaT_d = {16:5 , 17:3 , 18:1}
alphaN_nn = {nn:1 for nn in A}
betaN_nn = {nn:0 for nn in A}

BMC = 1

In [3]:
BM = Fmax
BMC = 1
BM1 = Fmax
BM2 = Fmax

R = S
C0hat_qr = CIN_qs
for n in B:
    if I0_n[n] != 0:
        R.append(n)
        
for q in Q:
    for n in B:
        if I0_n[n] != 0:
            C0hat_qr.update(C0_qb)

# Model
def master():
    m1 = Model("Master")
    m1.setParam('OutputFlag', 1)
    m1.setParam('TimeLimit', 3*60*60)
    m1.setParam('MIPGap', 0.0001)

    #Variables
    F = {(nn,t):m1.addVar(vtype=GRB.CONTINUOUS,name="F_"+str((nn,t))) for nn in A for t in T}
    FD = {(d,t):m1.addVar(vtype=GRB.CONTINUOUS,name="FD_"+str((d,t))) for d in D for t in T}
    I = {(n,t):m1.addVar(vtype=GRB.CONTINUOUS,name="I_"+str((n,t))) for n in N for t in T}
    C = {(q,b,t):m1.addVar(vtype=GRB.CONTINUOUS,name="C_"+str((q,b,t))) for q in Q for b in B for t in T}
    X = {(nn,t):m1.addVar(vtype=GRB.BINARY,name="X_"+str((nn,t))) for nn in A for t in T}
    YB = {(b,t):m1.addVar(vtype=GRB.BINARY,name="YB_"+str((b,t))) for b in B for t in T}
    Z = m1.addVar(vtype=GRB.CONTINUOUS,name="Z")

    #Redundant Variables
    Ftil = {(r,nn,t):m1.addVar(vtype=GRB.CONTINUOUS,name="Ftil_"+str((r,nn,t))) for r in R for nn in A for t in T}
    Itil = {(r,b,t):m1.addVar(vtype=GRB.CONTINUOUS,name="Itil_"+str((r,b,t))) for r in R for b in B for t in T}


    for t in T:
        for s in S: 
            if t==1:
                m1.addConstr(I[s,t] == I0_n[s] + FIN_st[s,t] - quicksum(F[(s,n),t] for n in B1) - quicksum(F[(s,d),t] for d in D))
            else:
                m1.addConstr(I[s,t] == I[s,t-1] + FIN_st[s,t] - quicksum(F[(s,n),t] for n in B1) - quicksum(F[(s,d),t] for d in D))
    

        for d in D: 
            if t==1:
                m1.addConstr(I[d,t] == I0_n[d] + quicksum(F[(n,d),t] for n in B2) + quicksum(F[(s,d),t] for s in S) - FD[d,t])
            else:
                m1.addConstr(I[d,t] == I[d,t-1] + quicksum(F[(n,d),t] for n in B2) + quicksum(F[(s,d),t] for s in S) - FD[d,t])
                
        for nn in A:
            m1.addConstr(F[nn,t] == quicksum(Ftil[r,nn,t] for r in R))
        
        for b in B:
            m1.addConstr(I[b,t] == quicksum(Itil[r,b,t] for r in R))

#___
        for n in S:
            for b in B1:
                m1.addConstr(FL_nn[(n,b)] - F[(n,b),t] <= BM * (1 - X[(n,b),t]))
                m1.addConstr(F[(n,b),t] - FU_nn[(n,b)] <= BM * (1 - X[(n,b),t]))
                m1.addConstr(F[(n,b),t] <= BM * X[(n,b),t])

        for n in B1:
            for b in B2:
                m1.addConstr(FL_nn[(n,b)] - F[(n,b),t] <= BM * (1 - X[(n,b),t]))
                m1.addConstr(F[(n,b),t] - FU_nn[(n,b)] <= BM * (1 - X[(n,b),t]))
                m1.addConstr(F[(n,b),t] <= BM * X[(n,b),t])
#___            
        for s in S:
            for d in D:
                m1.addConstr(FL_nn[(s,d)] - F[(s,d),t] <= BM * (1 - X[(s,d),t]))
                m1.addConstr(F[(s,d),t] - FU_nn[(s,d)] <= BM * (1 - X[(s,d),t]))
                m1.addConstr(F[(s,d),t] <= BM * X[(s,d),t])
                for q in Q:
                    m1.addConstr(CL_qd[(q,d)] - CIN_qs[q,s] <= BMC * (1 - X[(s,d),t]))
                    m1.addConstr(CIN_qs[q,s] - CU_qd[(q,d)] <= BMC * (1 - X[(s,d),t]))
                
#___           
        for b in B2:
            for d in D:
                m1.addConstr(FL_nn[(b,d)] - F[(b,d),t] <= BM * (1 - X[(b,d),t]))
                m1.addConstr(F[(b,d),t] - FU_nn[(b,d)] <= BM * (1 - X[(b,d),t]))
                m1.addConstr(F[(b,d),t] <= BM * X[(b,d),t])
                for q in Q:
                    m1.addConstr(CL_qd[(q,d)] * F[(b,d),t] - quicksum(Ftil[r,(b,d),t] * C0hat_qr[q,r] for r in R) <= BM2 * (1 - X[(b,d),t]))
                    m1.addConstr(quicksum(Ftil[r,(b,d),t] * C0hat_qr[q,r] for r in R) - CU_qd[(q,d)] * F[(b,d),t] <= BM2 * (1 - X[(b,d),t]))
                    if t > 1:
                        m1.addConstr(CL_qd[(q,d)] - C[q,b,t-1] <= BMC * (1 - X[(b,d),t]))
                        m1.addConstr(C[q,b,t-1] - CU_qd[(q,d)] <= BMC * (1 - X[(b,d),t]))                   
                        m1.addConstr(CL_qd[(q,d)] * I[b,t-1] - quicksum(Itil[r,b,t-1] * C0hat_qr[q,r] for r in R) <= BM2 * (1 - X[(b,d),t]))
                        m1.addConstr(quicksum(Itil[r,b,t-1] * C0hat_qr[q,r] for r in R) - CU_qd[(q,d)] * I[b,t-1] <= BM2 * (1 - X[(b,d),t]))
#___
        for b in B1:
            if t==1:
                m1.addConstr(I[b,t] <= I0_n[b] + quicksum(F[(s,b),t] for s in S) + BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] >= I0_n[b] + quicksum(F[(s,b),t] for s in S) - BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] <= I0_n[b] - quicksum(F[(b,n),t] for n in B2) + BM2 * YB[b,t])
                m1.addConstr(I[b,t] >= I0_n[b] - quicksum(F[(b,n),t] for n in B2) - BM2 * YB[b,t])

            else:
                m1.addConstr(I[b,t] <= I[b,t-1] + quicksum(F[(s,b),t] for s in S) + BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] >= I[b,t-1] + quicksum(F[(s,b),t] for s in S) - BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] <= I[b,t-1] - quicksum(F[(b,n),t] for n in B2) + BM2 * YB[b,t])
                m1.addConstr(I[b,t] >= I[b,t-1] - quicksum(F[(b,n),t] for n in B2) - BM2 * YB[b,t])

                
            for q in Q:
                if t > 1:
                    m1.addConstr(C[q,b,t] <= C[q,b,t-1] + BMC * YB[b,t])
                    m1.addConstr(C[q,b,t] >= C[q,b,t-1] - BMC * YB[b,t])
                
            for r in R:
                if t==1:
                    m1.addConstr(Itil[r,b,t] <= I0_n[b] + quicksum(Ftil[r,(s,b),t] for s in S) + BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] >= I0_n[b] + quicksum(Ftil[r,(s,b),t] for s in S) - BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] <= I0_n[b] - quicksum(Ftil[r,(b,n),t] for n in B2) + BM2 * YB[b,t])
                    m1.addConstr(Itil[r,b,t] >= I0_n[b] - quicksum(Ftil[r,(b,n),t] for n in B2) - BM2 * YB[b,t])
                else:
                    m1.addConstr(Itil[r,b,t] <= Itil[r,b,t-1] + quicksum(Ftil[r,(s,b),t] for s in S) + BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] >= Itil[r,b,t-1] + quicksum(Ftil[r,(s,b),t] for s in S) - BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] <= Itil[r,b,t-1] - quicksum(Ftil[r,(b,n),t] for n in B2) + BM2 * YB[b,t])
                    m1.addConstr(Itil[r,b,t] >= Itil[r,b,t-1] - quicksum(Ftil[r,(b,n),t] for n in B2) - BM2 * YB[b,t])
                
                
        for b in B2:
            if t==1:
                m1.addConstr(I[b,t] <= I0_n[b] + quicksum(F[(n,b),t] for n in B1) + BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] >= I0_n[b] + quicksum(F[(n,b),t] for n in B1) - BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] <= I0_n[b] - quicksum(F[(b,d),t] for d in D) + BM2 * YB[b,t])
                m1.addConstr(I[b,t] >= I0_n[b] - quicksum(F[(b,d),t] for d in D) - BM2 * YB[b,t])
            else:
                m1.addConstr(I[b,t] <= I[b,t-1] + quicksum(F[(n,b),t] for n in B1) + BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] >= I[b,t-1] + quicksum(F[(n,b),t] for n in B1) - BM2 * (1 - YB[b,t]))
                m1.addConstr(I[b,t] <= I[b,t-1] - quicksum(F[(b,d),t] for d in D) + BM2 * YB[b,t])
                m1.addConstr(I[b,t] >= I[b,t-1] - quicksum(F[(b,d),t] for d in D) - BM2 * YB[b,t])
            
            for q in Q:
                if t > 1:
                    m1.addConstr(C[q,b,t] <= C[q,b,t-1] + BMC * YB[b,t])
                    m1.addConstr(C[q,b,t] >= C[q,b,t-1] - BMC * YB[b,t])
                
            for r in R:
                if t==1:
                    m1.addConstr(Itil[r,b,t] <= I0_n[b] + quicksum(Ftil[r,(n,b),t] for n in B1) + BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] >= I0_n[b] + quicksum(Ftil[r,(n,b),t] for n in B1) - BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] <= I0_n[b] - quicksum(Ftil[r,(b,d),t] for d in D) + BM2 * YB[b,t])
                    m1.addConstr(Itil[r,b,t] >= I0_n[b] - quicksum(Ftil[r,(b,d),t] for d in D) - BM2 * YB[b,t])
                else:
                    m1.addConstr(Itil[r,b,t] <= Itil[r,b,t-1] + quicksum(Ftil[r,(n,b),t] for n in B1) + BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] >= Itil[r,b,t-1] + quicksum(Ftil[r,(n,b),t] for n in B1) - BM2 * (1 - YB[b,t]))
                    m1.addConstr(Itil[r,b,t] <= Itil[r,b,t-1] - quicksum(Ftil[r,(b,d),t] for d in D) + BM2 * YB[b,t])
                    m1.addConstr(Itil[r,b,t] >= Itil[r,b,t-1] - quicksum(Ftil[r,(b,d),t] for d in D) - BM2 * YB[b,t])
    
#___
        for b in B1:
            for n in S:
                m1.addConstr(X[(n,b),t] <= YB[b,t])
            
        for b in B2:
            for n in B1:
                m1.addConstr(X[(n,b),t] <= YB[b,t])
#___
        for b in B1:
            for n in B2:
                m1.addConstr(X[(b,n),t] <= 1 - YB[b,t])
            
        for b in B2:
            for n in D:
                m1.addConstr(X[(b,n),t] <= 1 - YB[b,t])
    

#Contraints paper K
        
               
    #6 Naturaleza de las variables
        for n in N:
            m1.addConstr(IL_n[n] <= I[n,t])
            m1.addConstr(I[n,t] <= IU_n[n])  
        
        for nn in A:   
            m1.addConstr(F[nn,t] >= 0)
                
        for d in D:
            m1.addConstr(FDL_dt[d,t] <= FD[d,t])
            m1.addConstr(FD[d,t] <= FDU_dt[d,t])
        
        for b in B:
            for q in Q:
                m1.addConstr(CL_q[q] <= C[q,b,t])
                m1.addConstr(C[q,b,t] <= CU_q[q])
    
        for r in R:
            for nn in A:
                m1.addConstr(Ftil[r,nn,t] <= FU_nn[nn]) 
                m1.addConstr(Ftil[r,nn,t] >= 0)
            
        for r in R:
            for s in S:
                for n in B1:
                    if r == s:
                        m1.addConstr(Ftil[r,(s,n),t] == F[(s,n),t])
#___   
        if t ==1:
            for r in R:
                for b in B1:
                    for n in B2:
                        if r == b:
                            m1.addConstr(Ftil[r,(b,n),t] == F[(b,n),t])
    
            for r in R:
                for b in B2:
                    for n in D:
                        if r == b:
                            m1.addConstr(Ftil[r,(b,n),t] == F[(b,n),t])
                        
        for s in S:
            for d in D:
                m1.addConstr(F[(s,d),t] == 0)
#__
    m1. addConstr(Z <= quicksum(quicksum(betaT_d[d] * F[(n,d),t] for d in D for n in B2) + quicksum(betaT_d[d] * F[(s,d),t] for d in D for s in S) -quicksum(alphaN_nn[nn] * X[nn,t] + betaN_nn[nn] * F[nn,t] for nn in A) for t in T))        
    m1.update()
    OF = m1.setObjective(Z,GRB.MAXIMIZE)
    tstart = time.time()
    m1.optimize()

    it = 1
    cuts = 0    
    LB, UB, CUT, IO, IF = [1], [2000], [0], [], []
    epsilon = 0.0001
    
    while True:
        UB.append(m1.objVal)
        paint(F,C,'master')
        
        ybfix = {(b,t):YB[b,t].x for b in B for t in T} 
        
        for t in T:
            for b in B1:
                if (sum(X[(n,b),t].x for n in S) + sum(X[(b,n),t].x for n in B2)) == 0:
                    ybfix[b,t] = 0
        
        for t in T:
            for b in B2:
                if (sum(X[(n,b),t].x for n in B1) + sum(X[(b,n),t].x for n in D)) == 0:
                    ybfix[b,t] = 0
        print()
        
        Zi = []

        print('Empiezo SP')
        Zi_star, sp_status  = subproblem(ybfix)
        
        if sp_status == 2:
            
            Zi.append(Zi_star)
            
            if Zi[-1] > LB[-1]:
                LB.append(Zi[-1])

        gap = (UB[-1] - LB[-1])/LB[-1]   
        if gap <= epsilon:
            return LB[-1], UB[-1], round(gap*100,3),it
            break
            
        else:
            if sp_status == 2:
                print('Hago corte de optimalidad')
                m1.addConstr(Z <= -(UB[-1] - Zi[-1])*(quicksum(YB[b,t] for b in B for t in T if ybfix[b,t]==1) - quicksum(YB[b,t] for b in B for t in T if ybfix[b,t]==0)) + (UB[-1] - Zi[-1])*(quicksum(ybfix[b,t] for b in B for t in T) - 1) + UB[-1])    
                m1.update()
                IO.append(it)
                cuts += 1
            
            elif sp_status == 3: 
                print('Hago corte de factibilidad')
                m1.addConstr(quicksum(1 - YB[b,t] for b in B for t in T if ybfix[b,t]==1) + quicksum(YB[b,t] for b in B for t in T if ybfix[b,t] == 0) >= 1)
                m1.update()
                IF.append(it)
                cuts += 1
            
            else:
                print('WTF')
            
            CUT.append(cuts)

            print('Vuelvo al master')
            m1.optimize()
        
        tend = time.time() - tstart
        
        if tend > 10800:
            return "Time Out","Tiempo", round(tend,2), "Gap", round(gap*100,3) ,"Iteraciones" ,it
            break

            
        it += 1
    
    return OF

In [4]:
def subproblem(ybfix):
    sp = Model("Subproblem")
    sp.setParam('OutputFlag', 1)
    sp.setParam('NonConvex', 2)
    sp.setParam('TimeLimit', 3*60*60)
    sp.setParam('MIQCPMethod', 1)
    sp.setParam('MIPGap', 0.0001)
    sp.setParam('DualReductions', 0)

    #Variables
    F = {(nn,t):sp.addVar(vtype=GRB.CONTINUOUS,name="F_"+str((nn,t))) for nn in A for t in T}
    FD = {(d,t):sp.addVar(vtype=GRB.CONTINUOUS,name="FD_"+str((d,t))) for d in D for t in T}
    I = {(n,t):sp.addVar(vtype=GRB.CONTINUOUS,name="I_"+str((n,t))) for n in N for t in T}
    C = {(q,b,t):sp.addVar(vtype=GRB.CONTINUOUS,name="C_"+str((q,b,t))) for q in Q for b in B for t in T}
    X = {(nn,t):sp.addVar(vtype=GRB.BINARY,name="X_"+str((nn,t))) for nn in A for t in T}
    #YB = {(b,t):sp.addVar(vtype=GRB.BINARY,name="YB_"+str((b,t))) for b in B for t in T}
    Z = sp.addVar(vtype=GRB.CONTINUOUS,name="Z")

    #Redundant Variables
    Ftil = {(r,nn,t):sp.addVar(vtype=GRB.CONTINUOUS,name="Ftil_"+str((r,nn,t))) for r in R for nn in A for t in T}
    Itil = {(r,b,t):sp.addVar(vtype=GRB.CONTINUOUS,name="Itil_"+str((r,b,t))) for r in R for b in B for t in T}

    for b in B1:
        for t in T:
            if ybfix[b,t] == 1:
                for n in B2:
                    sp.remove(F[(b,n),t])
                    sp.remove(X[(b,n),t])
                    for r in R:
                        sp.remove(Ftil[r,(b,n),t])
            else:
                for n in S:
                    sp.remove(F[(n,b),t])
                    sp.remove(X[(n,b),t])
                    for r in R:
                        sp.remove(Ftil[r,(n,b),t])
                    
    for b in B2:
        for t in T:
            if ybfix[b,t] == 1:
                for n in D:
                    sp.remove(F[(b,n),t])
                    sp.remove(X[(b,n),t])
                    for r in R:
                        sp.remove(Ftil[r,(b,n),t])
            else:
                for n in B1:
                    sp.remove(F[(n,b),t])
                    sp.remove(X[(n,b),t])
                    for r in R:
                        sp.remove(Ftil[r,(n,b),t])

    for t in T:
        for s in S: 
            if t==1:
                sp.addConstr(I[s,t] == I0_n[s] + FIN_st[s,t] - quicksum(F[(s,n),t] for n in B1) - quicksum(F[(s,d),t] for d in D))
            else:
                sp.addConstr(I[s,t] == I[s,t-1] + FIN_st[s,t] - quicksum(F[(s,n),t] for n in B1) - quicksum(F[(s,d),t] for d in D))
    

        for d in D: 
            if t==1:
                sp.addConstr(I[d,t] == I0_n[d] + quicksum(F[(n,d),t] for n in B2) + quicksum(F[(s,d),t] for s in S) - FD[d,t])
            else:
                sp.addConstr(I[d,t] == I[d,t-1] + quicksum(F[(n,d),t] for n in B2) + quicksum(F[(s,d),t] for s in S) - FD[d,t])
                
        for nn in A:
            sp.addConstr(F[nn,t] == quicksum(Ftil[r,nn,t] for r in R))
        
        for b in B:
            sp.addConstr(I[b,t] == quicksum(Itil[r,b,t] for r in R))

#___
        for n in S:
            for b in B1:
                if ybfix[b,t] == 1:
                    sp.addConstr(FL_nn[(n,b)] - F[(n,b),t] <= BM * (1 - X[(n,b),t]))
                    sp.addConstr(F[(n,b),t] - FU_nn[(n,b)] <= BM * (1 - X[(n,b),t]))
                    sp.addConstr(F[(n,b),t] <= BM * X[(n,b),t])

        for n in B1:
            for b in B2:
                if ybfix[b,t] == 1:
                    sp.addConstr(FL_nn[(n,b)] - F[(n,b),t] <= BM * (1 - X[(n,b),t]))
                    sp.addConstr(F[(n,b),t] - FU_nn[(n,b)] <= BM * (1 - X[(n,b),t]))
                    sp.addConstr(F[(n,b),t] <= BM * X[(n,b),t])
#___            
        for s in S:
            for d in D:
                sp.addConstr(FL_nn[(s,d)] - F[(s,d),t] <= BM * (1 - X[(s,d),t]))
                sp.addConstr(F[(s,d),t] - FU_nn[(s,d)] <= BM * (1 - X[(s,d),t]))
                sp.addConstr(F[(s,d),t] <= BM * X[(s,d),t])
                for q in Q:
                    sp.addConstr(CL_qd[(q,d)] - CIN_qs[q,s] <= BMC * (1 - X[(s,d),t]))
                    sp.addConstr(CIN_qs[q,s] - CU_qd[(q,d)] <= BMC * (1 - X[(s,d),t]))
                
#___           
        for b in B2:
            for d in D:
                if ybfix[b,t] == 0:
                    sp.addConstr(FL_nn[(b,d)] - F[(b,d),t] <= BM * (1 - X[(b,d),t]))
                    sp.addConstr(F[(b,d),t] - FU_nn[(b,d)] <= BM * (1 - X[(b,d),t]))
                    sp.addConstr(F[(b,d),t] <= BM * X[(b,d),t])
                    for q in Q:
                        sp.addConstr(CL_qd[(q,d)] * F[(b,d),t] - quicksum(Ftil[r,(b,d),t] * C0hat_qr[q,r] for r in R) <= BM2 * (1 - X[(b,d),t]))
                        sp.addConstr(quicksum(Ftil[r,(b,d),t] * C0hat_qr[q,r] for r in R) - CU_qd[(q,d)] * F[(b,d),t] <= BM2 * (1 - X[(b,d),t]))
                        if t > 1:
                            sp.addConstr(CL_qd[(q,d)] - C[q,b,t-1] <= BMC * (1 - X[(b,d),t]))
                            sp.addConstr(C[q,b,t-1] - CU_qd[(q,d)] <= BMC * (1 - X[(b,d),t]))                   
                            sp.addConstr(CL_qd[(q,d)] * I[b,t-1] - quicksum(Itil[r,b,t-1] * C0hat_qr[q,r] for r in R) <= BM2 * (1 - X[(b,d),t]))
                            sp.addConstr(quicksum(Itil[r,b,t-1] * C0hat_qr[q,r] for r in R) - CU_qd[(q,d)] * I[b,t-1] <= BM2 * (1 - X[(b,d),t]))
#___
        for b in B1:
            if ybfix[b,t] == 1:
                if t==1:
                    sp.addConstr(I[b,t] == I0_n[b] + quicksum(F[(s,b),t] for s in S))
                else:
                    sp.addConstr(I[b,t] == I[b,t-1] + quicksum(F[(s,b),t] for s in S))
    
        for b in B2:
            if ybfix[b,t] == 1:
                if t==1:
                    sp.addConstr(I[b,t] == I0_n[b] + quicksum(F[(n,b),t] for n in B1))
                else:
                    sp.addConstr(I[b,t] == I[b,t-1] + quicksum(F[(n,b),t] for n in B1))
#___
        for q in Q:
            for b in B1:
                if ybfix[b,t] == 1:
                    if t==1:
                        sp.addConstr(I[b,t] * C[q,b,t] == I0_n[b] * C0_qb[q,b] + quicksum(F[(s,b),t] * CIN_qs[q,s]  for s in S))
                    else:
                        sp.addConstr(I[b,t] * C[q,b,t] == I[b,t-1] * C[q,b,t-1] + quicksum(F[(s,b),t] * CIN_qs[q,s]  for s in S))
                
        for q in Q:
            for b in B2:
                if ybfix[b,t] == 1:
                    if t==1:
                        sp.addConstr(I[b,t] * C[q,b,t] == I0_n[b] * C0_qb[q,b] + quicksum(F[(n,b),t] * C0_qb[q,n]  for n in B1))
                    else:
                        sp.addConstr(I[b,t] * C[q,b,t] == I[b,t-1] * C[q,b,t-1] + quicksum(F[(n,b),t] * C[q,n,t-1]  for n in B1 if ybfix[n,t] == 0))
#___
        for r in R:
            for b in B1:
                if ybfix[b,t] == 1:
                    if t==1:
                        sp.addConstr(Itil[r,b,t] == I0_n[b] + quicksum(Ftil[r,(n,b),t] for n in S))
                    else:
                        sp.addConstr(Itil[r,b,t] == Itil[r,b,t-1] + quicksum(Ftil[r,(n,b),t] for n in S))
                        
        for r in R:
            for b in B2:
                if ybfix[b,t] == 1:
                    if t==1:
                        sp.addConstr(Itil[r,b,t] == I0_n[b] + quicksum(Ftil[r,(n,b),t] for n in B1))
                    else:
                        sp.addConstr(Itil[r,b,t] == Itil[r,b,t-1] + quicksum(Ftil[r,(n,b),t] for n in B1))      
#___
        for b in B1:
            if ybfix[b,t] == 0:
                if t==1:
                    sp.addConstr(I[b,t] == I0_n[b] - quicksum(F[(b,n),t] for n in B2))
                else:
                    sp.addConstr(I[b,t] == I[b,t-1] - quicksum(F[(b,n),t] for n in B2))
                        
        for b in B2:
            if ybfix[b,t] == 0:
                if t==1:
                    sp.addConstr(I[b,t] == I0_n[b] - quicksum(F[(b,n),t] for n in D))
                else:
                    sp.addConstr(I[b,t] == I[b,t-1] - quicksum(F[(b,n),t] for n in D)) 
#___
        for q in Q:
            for b in B:
                if ybfix[b,t] == 0:
                    if t > 1:
                        sp.addConstr(C[q,b,t] == C[q,b,t-1])
#___
        for r in R:
            for b in B1:
                if ybfix[b,t] == 0:
                    if t==1:
                        sp.addConstr(Itil[r,b,t] == I0_n[b] - quicksum(Ftil[r,(b,n),t] for n in B2))
                    else:
                        sp.addConstr(Itil[r,b,t] == Itil[r,b,t-1] - quicksum(Ftil[r,(b,n),t] for n in B2))
                        
        for r in R:
            for b in B2:
                if ybfix[b,t] == 0:
                    if t==1:
                        sp.addConstr(Itil[r,b,t] == I0_n[b] - quicksum(Ftil[r,(b,n),t] for n in D))
                    else:
                        sp.addConstr(Itil[r,b,t] == Itil[r,b,t-1] - quicksum(Ftil[r,(b,n),t] for n in D))                    
#___
        for b in B2:
            for d in D:
                if ybfix[b,t] == 1:
                    sp.addConstr(F[(b,d),t] == 0)
#___
        for b in B1:
            for n in S:
                if ybfix[b,t] ==0:
                    sp.addConstr(F[(n,b),t] == 0)
                    
        for b in B2:
            for n in B1:
                if ybfix[b,t] == 0:
                    sp.addConstr(F[(n,b),t] == 0)                
        
               
    #6 Naturaleza de las variables
        for n in N:
            sp.addConstr(IL_n[n] <= I[n,t])
            sp.addConstr(I[n,t] <= IU_n[n])  
        
        for nn in A:   
            sp.addConstr(F[nn,t] >= 0)
                
        for d in D:
            sp.addConstr(FDL_dt[d,t] <= FD[d,t])
            sp.addConstr(FD[d,t] <= FDU_dt[d,t])

        
        for b in B:
            for q in Q:
                sp.addConstr(CL_q[q] <= C[q,b,t])
                sp.addConstr(C[q,b,t] <= CU_q[q])
    
        for r in R:
            for nn in A:
                sp.addConstr(Ftil[r,nn,t] <= FU_nn[nn]) 
                sp.addConstr(Ftil[r,nn,t] >= 0)
            
        for r in R:
            for s in S:
                for n in B1:
                    if r == s:
                        sp.addConstr(Ftil[r,(s,n),t] == F[(s,n),t])
#___   

        for r in R:
            for b in B1:
                for n in B2:
                    if r == b:
                        sp.addConstr(Ftil[r,(b,n),t] == F[(b,n),t])
    
        for r in R:
            for b in B2:
                for n in D:
                    if r == b:
                        sp.addConstr(Ftil[r,(b,n),t] == F[(b,n),t])
                        
        for s in S:
            for d in D:
                sp.addConstr(F[(s,d),t] == 0)
                                        
#__
    sp.addConstr(Z == quicksum(quicksum(betaT_d[d] * F[(n,d),t] for d in D for n in B2) + quicksum(betaT_d[d] * F[(s,d),t] for d in D for s in S) - quicksum(alphaN_nn[nn] * X[nn,t] + betaN_nn[nn] * F[nn,t] for nn in A) for t in T))        
    sp.update()
    OF = sp.setObjective(Z,GRB.MAXIMIZE)
    sp.optimize()
    
    if sp.status == 2:
        paint(F,C,'subproblem')
        return sp.objVal, sp.status
    else:
        return -9999, sp.status
   
    

In [5]:
def paint(F,C,problem):
    
    if len(N) == 8:
        x = [1, 1, 2, 2, 3, 3, 4, 4]
        y = [3, 2, 4, 1, 4, 1, 3, 2]
    elif len(N) == 12:
        x = [1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4]
        y = [3, 2, 4, 3, 2, 1, 4, 3, 2, 1, 3, 2]
    elif len(N) == 18:
        x = [1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4]
        y = [-2.5, -3.5, -4.5, -1, -2, -3, -4, -5, -6, -1, -2, -3, -4, -5, -6, -2.5, -3.5, -4.5]

    pos = {(i+1):(x[i],y[i]) for i in range(len(N))}

    for t in T:
        print('Los flujos a operar en el tiempo',t, 'son:')
        arcs = []
        flows = []
        for nn in A:
            try:
                if F[nn,t].x >= 0.01:
                    arcs.append(((nn)))
                    flows.append(round(F[nn,t].x,2))
            except:
                pass
            
        graph = nx.DiGraph()
        for i in range(len(arcs)):
            graph.add_edge(arcs[i][0],arcs[i][1])
 
        pairs = list(zip(list(arcs),list(flows)))
        edgelabels = dict(pairs)
    
        nodelabels = {i:str(i) for i in range(1,len(N)+1)}
    
        if problem == 'master':
            colorea = 'skyblue'
        else:
            colorea = 'salmon'
        
        nx.draw_networkx(graph,pos,node_size=700,node_color=colorea,width=1.5,nodelist=list(range(1,len(N)+1)), with_labels=True)
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edgelabels)
        nx.draw_networkx_labels(graph,pos,nodelabels)
        plt.show()
        if t > 1:
            for d in D:
                for b2 in B2:
                    try:
                        if F[(b2,d),t].x >= 0.1:
                            for q in Q:
                                print('La concentración que salio por',d,'es',round(C[q,b2,t-1].x,2),'que debe estar entre',CL_qd[q,d],'y',CU_qd[q,d])
                    except:
                        pass
        print() 

In [ ]:
master()

Using license file C:\Users\danie\gurobi.lic
Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter TimeLimit to 10800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Parameter MIPGap unchanged
   Value: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 8617 rows, 2917 columns and 22627 nonzeros
Model fingerprint: 0x2cf01319
Variable types: 2359 continuous, 558 integer (558 binary)
Coefficient statistics:
  Matrix range     [5e-02, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 8e+01]
Presolve removed 5961 rows and 1158 columns
Presolve time: 0.14s
Presolved: 2656 rows, 1759 columns, 11197 nonzeros
Variable types: 1387 continuous, 372 integer (372 binary)

Root relaxation: objective 1.415000e+03, 625 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objecti

 13328  9285  871.41667   29   58  631.00000 1110.56447  76.0%   147  191s
 13711  9517  675.36667   36   37  631.00000 1107.93313  75.6%   147  195s
 14052  9575  997.00000   27  100  631.00000 1107.67103  75.5%   148  201s
 14111  9925  887.80556   30   65  631.00000 1107.65125  75.5%   148  206s
 14591 10237  644.00000   42   33  631.00000 1107.09333  75.5%   148  210s
 15481 10821 1052.11091   26  107  631.00000 1104.83333  75.1%   148  219s
 15883 11126 1033.50000   20   79  631.00000 1104.49272  75.0%   149  223s
 16320 11474  978.80534   27   93  631.00000 1103.66667  74.9%   149  227s
 16779 11714  789.29841   24   82  631.00000 1103.18627  74.8%   149  231s
 17122 12061  741.83333   33   51  631.00000 1101.55285  74.6%   149  236s
 17595 12361  634.00000   37   31  631.00000 1097.83333  74.0%   149  240s
 18563 12943  772.82639   30   51  631.00000 1093.91852  73.4%   148  248s
 18951 13186  634.33333   38   24  631.00000 1091.92361  73.0%   149  253s
 19321 13441  919.73333  

 64868  5906  633.00000   80   14  632.00000  633.00000  0.16%   249  789s
 65450  5897  633.00000   89   26  632.00000  633.00000  0.16%   250  795s
 66073  5702  633.00000   75   47  632.00000  633.00000  0.16%   251  801s
 66607  5661  633.00000   88   19  632.00000  633.00000  0.16%   252  807s
 67446  5468 infeasible   99       632.00000  633.00000  0.16%   253  813s
 68048  5292  633.00000   74   25  632.00000  633.00000  0.16%   253  818s
 68435  5168  633.00000   78   40  632.00000  633.00000  0.16%   255  823s
 68816  5089  633.00000   65   25  632.00000  633.00000  0.16%   256  830s
 69231  5011 infeasible   76       632.00000  633.00000  0.16%   257  835s
 69843  4956 infeasible   96       632.00000  633.00000  0.16%   258  841s
 70616  4739     cutoff   86       632.00000  633.00000  0.16%   258  847s
 71095  4689  633.00000   89   20  632.00000  633.00000  0.16%   259  853s
 71687  4528  633.00000   82   15  632.00000  633.00000  0.16%   260  859s
 72171  4413  633.00000  

 125542  7454 infeasible   72       632.00000  633.00000  0.16%   333 1543s
 125999  7463 infeasible   77       632.00000  633.00000  0.16%   334 1551s
 126558  7525 infeasible   72       632.00000  633.00000  0.16%   335 1559s
 127205  7522  633.00000   64   29  632.00000  633.00000  0.16%   335 1567s
 127766  7570  633.00000   68   35  632.00000  633.00000  0.16%   336 1575s
 128533  7661 infeasible   75       632.00000  633.00000  0.16%   336 1583s
 129224  7700 infeasible   72       632.00000  633.00000  0.16%   336 1591s
 129789  7706  633.00000   72   19  632.00000  633.00000  0.16%   337 1599s
 130257  7727  633.00000   74   20  632.00000  633.00000  0.16%   337 1607s
 130856  7765 infeasible   91       632.00000  633.00000  0.16%   338 1615s
 131580  7822  633.00000   88   27  632.00000  633.00000  0.16%   338 1623s
 132609  7860  633.00000   72   21  632.00000  633.00000  0.16%   338 1631s
 133228  7869 infeasible   90       632.00000  633.00000  0.16%   339 1639s
 133903  787

 195170  8151 infeasible   88       632.00000  633.00000  0.16%   335 2254s
 195813  8145  633.00000   95   18  632.00000  633.00000  0.16%   335 2260s
 196359  8139 infeasible   95       632.00000  633.00000  0.16%   335 2266s
 196897  8151 infeasible   90       632.00000  633.00000  0.16%   335 2271s
 197367  8151 infeasible   75       632.00000  633.00000  0.16%   335 2276s
 197963  8158  633.00000   92   19  632.00000  633.00000  0.16%   335 2282s
 198618  8148 infeasible   92       632.00000  633.00000  0.16%   335 2288s
 199272  8126 infeasible   92       632.00000  633.00000  0.16%   335 2294s
 199862  8145 infeasible   92       632.00000  633.00000  0.16%   334 2299s
 200371  8142     cutoff   87       632.00000  633.00000  0.16%   334 2305s
 201074  8130 infeasible   92       632.00000  633.00000  0.16%   334 2310s
 201670  8144  633.00000   83   15  632.00000  633.00000  0.16%   334 2316s
 202228  8128 infeasible   76       632.00000  633.00000  0.16%   334 2322s
 202796  813

 256863  8154 infeasible   73       632.00000  633.00000  0.16%   331 2900s
 257046  8171  633.00000   90   21  632.00000  633.00000  0.16%   331 2906s
 257587  8157 infeasible   74       632.00000  633.00000  0.16%   332 2913s
 258109  8176  633.00000   71   16  632.00000  633.00000  0.16%   332 2920s
 258558  8190  633.00000   85   15  632.00000  633.00000  0.16%   332 2928s
 259154  8183 infeasible   73       632.00000  633.00000  0.16%   332 2934s
 259581  8179  633.00000   73   12  632.00000  633.00000  0.16%   332 2942s
 260057  8164 infeasible   88       632.00000  633.00000  0.16%   332 2949s
 260736  8151  633.00000   89   23  632.00000  633.00000  0.16%   332 2956s
 261343  8178 infeasible   72       632.00000  633.00000  0.16%   332 2963s
 261943  8181 infeasible   72       632.00000  633.00000  0.16%   332 2970s
 262448  8177 infeasible   82       632.00000  633.00000  0.16%   333 2977s
 263020  8191  633.00000   72   20  632.00000  633.00000  0.16%   333 2984s
 263728  819

 315767 11442 infeasible   69       632.00000  633.00000  0.16%   350 3842s
 316356 11483  633.00000   70   23  632.00000  633.00000  0.16%   351 3853s
